# Optieprijzen, Taak 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

In dit werk bereken we numeriek een optieprijs op basis van de waarde van het onderliggende $S_0$ , de strike $K$, een waarde voor de volatiliteit $\sigma$ , de 'rente' $r$ en het aantal stapjes op de binominale boom $N$.

### Programma 1: Eindige Discretiesatie Methode

De volgende functie gebruiken we om van een vector met lengte $n$ een stapje terug te zetten op de binominale boom naar een vector met lengte $n-1$. In deze vector is $v[i]$ gelijk aan de 'grootste waarde' van de optie op de boom.

In [2]:
def stapje_terug(u,d,p,r,dt,V):
    # neemt de laatst berekende eindvector met lengte n en geeft een vector terug met lengte n-1
    n=np.size(V)-1
    v=np.zeros(n)
    for i in range(n):
        v[i]=(p*V[i]+(1-p)*V[i+1])*np.exp(-r*dt)
    return v

In [3]:
def barriere_optieprijs(optie, S0, B, kind, u, d):
    #indien die prijs vh onderliggende op een moment t groter/kleiner is dan 0 wordt de optieprijs 0
    n_t=len(optie)   
    for i in range(len(optie)):
        S_t= S0*d**i *u**(n_t-i-1)
        if kind== 'call':
            if S_t>B:
                optie[i]=0
        elif kind == 'put':
            if S_t<B:
                optie[i]=0

De volgende functie berekent de waarde van de optie op t=0 adhv één functie voor zowel met als zonder barriëre. door barriëre True of False te geven kiezen we dit.
We kunnen ook array_out een bool geven en zo vinden we oa de binominale boom terug.
Met kind bedoelen we de string 'put' of 'call'

In [4]:
def discretisatie(N, T, S0, K, r, sigma, kind,  barrier, B, array_out):
    dt=T/N
    dx=sigma*np.sqrt(dt)
    
    u=np.exp(dx)
    d=np.exp(-dx)
    
    p= (np.exp(r*dt)-d)/(u-d)
    
    #onderliggende prijs op tn
    eindprijs=np.zeros(N+1) 
    for i in range(N+1):
        eindprijs[i]=S0*(d**(i))*(u**(N-i))
        
    #optieprijs op tN
    if kind=='call':
        if barrier ==  True:           
            eindprijs=np.array([0 if prijs > B else prijs for prijs in eindprijs])
            optieprijs=np.maximum(eindprijs-K,0)
        else:
            optieprijs=np.maximum(eindprijs-K,0)
    elif kind=='put':
        if barrier ==  True:           
            optieprijs=np.array([K-prijs if B<prijs<K else 0 for prijs in eindprijs])
        else:
            optieprijs=np.maximum(K-eindprijs,0)

    #in stapjes verdisconteren, tegelijkertijd slaan we de tussenstapjes op in 'store'
    
    store=list([optieprijs])

    while np.size(optieprijs)>1:
        optieprijs=stapje_terug(u,d,p,r,dt,optieprijs)        
        
        if barrier ==  True:
            barriere_optieprijs(optieprijs, S0, B, kind, u,d)
    
        store.append(list([optieprijs]))
        
    if array_out:
        return [optieprijs[0],store, eindprijs]
    else: 
        return optieprijs[0]

In [5]:
discretisatie(N=2, T=2, S0=50, K=48, r=0.05, sigma=0.3, kind='call', barrier=True, B=62, array_out=True)

[0.4522469990083574,
 [array([0., 2., 0.]), [array([0.        , 0.96976102])], [array([0.452247])]],
 array([ 0.       , 50.       , 27.4405818])]

### Programma 2: Sommeren over paden

Om het wat overzichtelijk te houden voor valuatie heb ik dit programma opgesplist in 2 programma's voor zonder of met barriëre.

In [6]:
def sum_paths(N, T, S0, K, r, sigma, kind):
    dt=T/N
    dx=sigma*np.sqrt(dt)
    
    u=np.exp(dx)
    d=np.exp(-dx)
    
    p= (np.exp(r*dt)-d)/(u-d)
    
    #verschillende (binaire) paden
    paths = itertools.product([0, 1], repeat=N)
    
    total_prob=0
    V_T=0
    
    for path in paths:
        N_up= path.count(1)
        N_down= N-N_up
        
        path_prob=p**N_up*(1-p)**N_down
        eind_prijs=S0*(d**N_down)*(u**N_up)
        
        #optieprijs op tn
        if kind=='call':
            optieprijs=np.maximum(eind_prijs-K,0)
        elif kind=='put':
            optieprijs=np.maximum(K-eind_prijs,0)

        V_T+=optieprijs*path_prob
        total_prob+=path_prob
    
    V_0=np.exp(-r*T)*V_T
    
    return V_0
    

In [7]:
sum_paths(N=5, T=10, S0=1, K=1.05, r=0.05, sigma=0.3, kind='call')

0.5168539556639589

In [15]:
def sum_paths_barriere(N, T, S0, K, r, sigma, kind, B,):
    dt=T/N
    dx=sigma*np.sqrt(dt)
    
    u=np.exp(dx)
    d=np.exp(-dx)
    
    p= (np.exp(r*dt)-d)/(u-d)
    
    #verschillende (binaire) paden
    paths = itertools.product([0, 1], repeat=N)
    

    V_T=0
    
    if kind == 'call':
        for path in paths:
            S_t=S0
            i=0

            #we overlopen elke stap in elk pad, elke step is ofwel 1 of 0 (up of down)
            for step in path:
                i+=1
                S_t=S_t*u**step*d**(1-step)

                # berekenen de prijs vh onderliggende op elk stapje in de tijd S_t 
                # en kijken of het groter/kleiner is dan de barriëre indien deze de barriëre overschrijdt op een stap 
                # dan kunnen we direct door naar het volgende pad in de lijst
                if i<N:
                    if S_t < B:
                        continue
                    else:
                        break
                else:
                    eindprijs=0 if S_t > B else S_t
                    if eindprijs !=0 :
                        optieprijs=np.maximum(eindprijs-K,0)

                        # elementen tellen uit een lijst is een 'lang' proces, daarom doen we dit alleen als 
                        # het pad daadwerkelijk meetelt

                        N_up= path.count(1)
                        N_down= N-N_up

                        path_prob=p**N_up*(1-p)**N_down
                        V_T+=optieprijs*path_prob
    
    else:
        for path in paths:
            S_t=S0
            i=0
            for step in path:
                i+=1
                #call
                S_t=S_t*u**step*d**(1-step)                
                if i<N:
                    if S_t > B:
                        continue
                    else:
                        break
                else:
                    optieprijs=K-S_t if B<S_t<K else 0

                    N_up= path.count(1)
                    N_down= N-N_up

                    path_prob=p**N_up*(1-p)**N_down
                    V_T+=optieprijs*path_prob        
    
    V_0=np.exp(-r*T)*V_T
    
    return V_0

In [17]:
sum_paths_barriere(N=2, T=2, S0=50, K=48 , r=0.05, sigma=0.3, kind='call', B=62)

0.4522469990083574

# Programma 3: Monte-Carlo Simulatie

Voor lange numerieke berekeningen gebruiken we het package 'numba'. Hierdoor werkt het programma tot 50x sneller. In het vorige programma konden we niet gebruik maken van dit package vanwege het commando itertools, numba is niet compatibel met dit package. 

In [18]:
from numba import jit

In [19]:
@jit(nopython=True)
def monte_carlo(n, N, T, S0, K, r, sigma, kind):
    dt=T/N
    dx=sigma*np.sqrt(dt)
    
    u=np.exp(dx)
    d=np.exp(-dx)
    
    p= (np.exp(r*dt)-d)/(u-d)
    
    x=0
    totale_prijs=0
    
    while x<n:
        
        random_array=np.random.rand(N)
        path=[1 if el <= p else 0 for el in random_array]
        
        N_up= path.count(1)
        N_down= N-N_up
        
        eind_prijs=S0*(d**N_down)*(u**N_up)
        
        #optieprijs op tn
        if kind=='call':
            optieprijs=np.maximum(eind_prijs-K,0)
        elif kind=='put':
            optieprijs=np.maximum(K-eind_prijs,0)

        
        totale_prijs+=optieprijs
        x+=1
    
    V_T=totale_prijs/n
    V_0=np.exp(-r*T)*V_T
    
    return V_0

In [20]:
monte_carlo(n=200,N=2, T=2, S0=50, K=48, r=0.05, sigma=0.3, kind='put')

4.836762027800452

In [21]:
@jit(nopython=True)
def monte_carlo_barriere(n, N, T, S0, K, r, sigma, kind, B):
    dt=T/N
    dx=sigma*np.sqrt(dt)
    
    u=np.exp(dx)
    d=np.exp(-dx)
    
    p= (np.exp(r*dt)-d)/(u-d)
    
    x=0
    totale_prijs=0
    
    if kind=='call':
        while x<n:
        
            random_array=np.random.rand(N)
            path=[1 if el <= p else 0 for el in random_array]

            S_t=S0
            i=0

            for step in path:
                i+=1
                S_t=S_t*u**step*d**(1-step)

                # berekenen de prijs vh onderliggende op elk stapje in de tijd S_t 
                # en kijken of het groter/kleiner is dan de barriëre
                if i<N:
                    if S_t < B:
                        continue
                    else:
                        break
                else:
                    eindprijs=0 if S_t > B else S_t
                    if eindprijs !=0 :
                        optieprijs=np.maximum(eindprijs-K,0)

                        # elementen tellen uit een lijst is een 'lang' proces, daarom doen we dit alleen als 
                        # het pad daadwerkelijk meetelt

                totale_prijs+=optieprijs
            x+=1

    else:
        while x<n:
            random_array=np.random.rand(N)
            path=[1 if el <= p else 0 for el in random_array]

            S_t=S0
            i=0
            for step in path:   
                i+=1
                #call
                S_t=S_t*u**step*d**(1-step)                
                if i<N:
                    if S_t > B:
                        continue
                    else:
                        break
                else:
                    optieprijs=K-S_t if B<S_t<K else 0

                    totale_prijs+=optieprijs
            x+=1
    
    V_T=totale_prijs/n
    V_0=np.exp(-r*T)*V_T
    
    return V_0

In [23]:
monte_carlo_barriere(n=100000,N=2, T=2, S0=50, K=48, r=0.05, sigma=0.3, kind='call', B=62)

0.4529797082171621